<div style="padding-top: 10px;  padding-bottom: 10px;">
  <img src="https://insightfactoryai.sharepoint.com/:i:/r/sites/insightfactory.ai/Shared%20Documents/E.%20Marketing/Company%20Logos%20and%20Style%20Guide/PNG/insightfactory.ai%20logo%20multiline%20reversed.png" alt='insightfactory.ai' width=150   style="display: block; margin: 0 auto" /> 
</div>

# Model Inference

***Summary of process:*** This notebook is used to do inference on the ML model from the engineered features.

***Input Tables:***  
- 

***Output Table:*** 
- 

Note: This is just the overview of the process, For details please review the notebook thoroughly

**Business Rules:** <br/>
\<Describe the Business Rules that are encapsulated in this Enrichment\>

**Dependencies:**<br/>
-

**Ownership:**<br/>
\<Indicate who owns this Enrichment ruleset\>


#### Modification Schedule

| Date | Who | Description |
| ---: | :--- | :--- |
| 2025-09-24 | Yifan Gu | Initial version. |


#### Insight Factory Notebook Preparation

**(Do not modify/delete the following cell)**

In [0]:
%pip install pandas scikit-learn mlflow-skinny[databricks]
%pip install -U lightgbm
%pip install -q databricks-feature-engineering
dbutils.library.restartPython()

In [0]:
%run "/InsightFactory/Helpers/ML Inference (Unity Catalog) Entry"

In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, f1_score,precision_recall_curve

from sklearn.feature_selection import RFECV
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from sklearn.model_selection import TimeSeriesSplit


# Notebook Start

### Input Parameters

All Notebook Parameters (if any) are contained in the dictionary variable 'params'.  There are two ways to get the individual parameter from params, in both cases the parameter name is case-sensitive:

  1) Use dot-notation - refer to the example below.

      params = { "Name": "Test", "Values": { "Title": "Results", "Results": [ { "Definition": "Core Sample", "Outcome": "Prospective" }, { "Definition": "Follow-up", "Outcome": "For review" } ] } }

      params.Name produces 'Test'<br/>
      params.Values.Title produces 'Results'<br/>
      params.Values.Results[0] produces { "Definition": "Core Sample", "Outcome": "Prospective" }<br/>
      params.Values.Results[1].Definition produces 'Follow-up'

  2) Use the search_dictionary function as follows:  var1 = search_dictionary(params, "parameter-name").  

      There is an optional third parameter to this function: value_to_return_if_not_found -  this is the value to return if the particular parameter is not found in params.<br/>
      **Note** that value_to_return_if_not_found can take on any type (string, int, boolean, struct, ..) e.g search_dictionary(params, "IncorrectlyNamedParameter", False) will return the boolean False if "IncorrectlyNamedParameter" is not found in params.

  3) Use the load_pickle_model function as follows: model = load_pickle_model(client, "model_location", "model_version").

      There is an optional fourth parameter to this function: pickle_location - The exact location of the pickle file in the particular storage volume.<br/>
      **Note** client is the MlFlowClient object and model location is the exact location of the model with 3 part schema as "Catalog.Schema.ModelName"


**CAUTION:** There is another dictionary variable, 'config', that contains all of the configuration sent to this Notebook.  In most cases, you will have no use for 'config' but if you choose to use 'config' in this Notebook, note the following:
- Access the individual parameters within config by using the search_dictionary function e.g. search_dictionary(config, "ParameterName").  Dot-notation access **does not apply** to 'config'.
- Heed this **WARNING** - The individual parameter names within 'config' are subject to change outside of your control which may break your code.
<br/><br/>

### Enrichment Results

Add the code you need to perform your enrichment/extract in cell(s) below until the 'Notebook End' cell.

####Important: 
- Ensure that the result is stored in a PySpark dataframe as
    - 'df_result' e.g. df_result = ... for the Inference Results for the Table 
    - 'df_inference_data' e.g. df_inference_data = ... for us to perform data at the exit of the model so you don't have to perform inference on model.
<br/><br/>

### Running this Notebook directly in Databricks

This Notebook can be run directly from your Databricks Workspace.  If the Notebook relies on Notebook Parameters, please read the following instructions:
1) Add this line of code to a cell at the top of your Notebook and run that cell.<br/>
   ```dbutils.widgets.text('ParametersJSON', '{ "ModelName":"name","ModelAlias":"alias","ModelVersion":"version","ModelSchema":"DatabaseName for Model", "NotebookParameters": { "param1": "value1", "param2": "value2" } }')```
2) This will add a Parameter to the Notebook.  Simply replace (or remove) the pre-canned parameters, 'param1', 'param2' and their values with your own.
3) When you have finished running this Notebook directly in Databricks, comment out the line of code you added or delete the cell entirely.    

In [0]:
## code your inference master data or df_result here ##

df = spark.table("`09ad024f-822f-48e4-9d9e-b5e03c1839a2`.feature_selection.total_predict_table")

num_cols = [
    'Wagon_Twist14m','Wagon_BounceFrt','Wagon_BounceRr','Wagon_BodyRockFrt','Wagon_BodyRockRr','Wagon_LP1',
    'Wagon_LP2','Wagon_LP3','Wagon_LP4','Wagon_Speed','Wagon_BrakeCylinder','Wagon_IntrainForce','Wagon_Acc1',
    'Wagon_Acc2','Wagon_Acc3','Wagon_Acc4','Wagon_Twist2m','Wagon_Acc1_RMS','Wagon_Acc2_RMS','Wagon_Acc3_RMS',
    'Wagon_Acc4_RMS','Wagon_Rail_Pro_L','Wagon_Rail_Pro_R','Wagon_SND','Wagon_VACC','Wagon_VACC_L','Wagon_VACC_R',
    'Wagon_Curvature','Wagon_Track_Offset','Wagon_SND_L','Wagon_SND_R','w_row_count','Tng_Tonnage'
]
gcols = ["Tc_BaseCode","Tc_BaseCode_Mapped","Tc_SectionBreakStartKM","Tc_r_date","Tc_p_key"]

pdf = df.toPandas()
pdf["Tc_r_date"] = pd.to_datetime(pdf["Tc_r_date"])
pdf["Wagon_RecordingDate"] = pd.to_datetime(pdf["Wagon_RecordingDate"])
pdf = pdf.sort_values(["Tc_r_date"] + gcols[:-1] + ["Wagon_RecordingDate"], kind="mergesort").reset_index(drop=True)

X_all = (pdf[num_cols]
         .apply(pd.to_numeric, errors="coerce")
         .replace([np.inf, -np.inf], np.nan)
         #.fillna(0.0)              
         .astype("double"))

In [0]:
import mlflow, mlflow.lightgbm, json, pandas as pd
from mlflow.tracking import MlflowClient
reg_name = f"{ml_catalog}.{model_schema_name}.{model_name}"
version = 3
model_uri = f"models:/{reg_name}/{version}"
clf = mlflow.lightgbm.load_model(model_uri)

run_id = MlflowClient().get_model_version(reg_name, str(version)).run_id
cfg_path = MlflowClient().download_artifacts(run_id, "rfe_lightgbm.json")
with open(cfg_path, "r") as f:
    cfg = json.load(f)
#feats = cfg["selected_feats"]
thr   = float(cfg["threshold"])

#X_new = pdf[feats]
X_new = X_all
pdf["p_row"] = clf.predict_proba(X_new)[:, 1]

import numpy as np
import pandas as pd

gcols = ["Tc_BaseCode","Tc_BaseCode_Mapped","Tc_SectionBreakStartKM","Tc_r_date","Tc_p_key"]

def aggregate_to_group(df_rows, method="noisyor", lambda_=0.05):
    age_days = (df_rows["Tc_r_date"] - df_rows["Wagon_RecordingDate"]).dt.days.clip(lower=0)
    w_time = np.exp(-lambda_ * age_days)

    def agg_fn(g):
        p = g["p_row"].values
        if method == "mean":
            P = float(np.mean(p))
        elif method == "wavg":
            P = float(np.average(p, weights=w_time[g.index]))
        elif method == "max":
            P = float(np.max(p))
        else:  # noisyor
            P = float(1.0 - np.prod(1.0 - p))
        return pd.Series({"probability": P})

    out = df_rows.groupby(gcols, sort=False, as_index=False).apply(agg_fn).reset_index(drop=True)
    return out

agg_pred = aggregate_to_group(pdf, method="noisyor", lambda_=0.05)

agg_pred["target"] = (agg_pred["probability"] >= thr).astype("int32")

agg_pred = agg_pred.rename(columns={"Tc_p_key": "p_key"})
keep_cols = [ "p_key", "probability", "target"]
agg_pred = agg_pred[keep_cols]

df_result = spark.createDataFrame(agg_pred)
display(df_result)


# Notebook End

**(Do not modify/delete the following cell)**

####Important: 
1) Ensure that the result is stored in a PySpark dataframe as
 - 'df_inference_data' e.g. df_inference_data = ... for Sending Inference data and performing inference on that.
 - 'df_result' e.g. df_result = ...  for Sending Results of the Inference performed by you.

In [0]:
%run "/InsightFactory/Helpers/ML Inference (Unity Catalog) Exit"

# Testing or Debugging Zone